# word2vec

## Вступление

Иногда бывает полезно уметь оценить, насколько два слова близки друг другу по смыслу. Для этого можно поставить каждому слову в соответствие какое-нибудь число или набор чисел (то есть вектор) - и определять расстояние уже между числами, а не словами. 

Первая идея, появившаяся у человечества по этому поводу - составить список всех слов, а затем в качестве присвоенного слову числа брать его номер в этом списке. Но у этого метода есть существенный недостаток. Иногда стоящие рядом в словаре слова действительно оказываются семантически близки ("учитель", "учительница"), но часто - это не так (к слову "бумага" по смыслу более близко слово "картон", а по списку - бумеранг").

Данная проблема была решена в 2013 году, когда чешский аспирант Томаш Миколов предложил за параметр "близости" слов принимать вероятность встретить их стоящими рядом в тексте. Это и есть основная идея метода word2vec.

## Препроцессинг

In [18]:
import re   # Импортировали библиотеку 
            # для использования регулярных выражений

import numpy as np

Прежде чем перейти к обучению модели (которой ещё нет, но скоро будет) на тексте, его нужно к этому подготовить:
    
    1. Провести токенизацию(т. е. разбить на отдельные слова)
    2. Удалить пунктуационные символы
    3. Привести все слова к нижнему регистру 
    
Это всё делает описанная ниже функция:

In [3]:
def data_generate(file):
    
    f = open(file, 'r')          # Открыли текстовый файл
    arr = []                     # Создали массив для последовательной записи слов текста
    
    for line in f:
        s = line
        s = s.lower()            # Привели все строчки к нижнему регистру 
                                 # (это понизит количество слов в словаре без ротери данных)
        
        spl = re.split('[^a-z]', s)
        k = spl.count('')
        for i in range(k):
            spl.remove('')       # Разбили строки на слова и удалили все пунктуационные символы
       
        for i in range(len(spl)):
            arr.append(spl[i])   # Заполнили массив словами текста (последовательно)        
     
    return(arr)

Теперь применим функцию к тексту, который мы будем использовать - отрывок из книги Марка Твена "Приключения Тома Сойера"

In [5]:
words = data_generate('training_text.txt')

Чтобы проверить, как сработала функция, посмотрим на первые 10 слов:

In [6]:
print words[:10]

['tom', 'joined', 'the', 'new', 'order', 'of', 'cadets', 'of', 'temperance', 'being']


Как видим, функция действительно работает - слова разбиты по-одному, пробелы и символы убраны, регистр везде нижний. Можно двигаться дальше.

Чтобы было удобнее обращаться к словам, создадим два словаря. В одном из них каждому слову будет присвоен порядковый номер, а в другом - пары ключ-значение будут поменяны местами (т. е. каждому номеру поставлено в соответствие слово). 

In [7]:
d_words = {}
d_indexes = {}

for i, word in enumerate(set(words)):
    d_words[word] = i
    d_indexes[i] = word

Посмотрим на количество слов в словаре:

In [13]:
voc_size = len(d_words)
print voc_size

457


Теперь можно заняться созданием элементов данных. Выберем размер контекстного окна window_size (это гиперпараметр, то есть параметр, который нельзя извлечь из самого процесса обучения). Для каждого слова будем определять контекстное окно размером window_size вправо и влево от целевого слова, а затем будем записывать все пары "целевое слово - слово из контекстного окна". Эти пары и будут нужными нам элементами данных.

In [8]:
window_size = 3

Слова из каждой пары мы будем записывать в два отдельных массива (точнее, вектора) в виде их индексов в словаре. В векторе X будут лежать целевые слова. Это одномерная матрица признаков, по которым модель должна предсказывать вероятность появления контекстных слов. В вектор Y положим контекстные слова - они будут ответами на обучающей выборке.

In [34]:
X = []
Y = []

for i in range(len(words)):
    for j in range(1, window_size+1):
        
        if i - j >= 0:
          
            X.append(d_words[words[i]])
            Y.append(d_words[words[i-j]])
            
            
        if i + j <= len(words)-1:
           
            X.append(d_words[words[i]])
            Y.append(d_words[words[i+j]])
            
X = np.array(X)
Y = np.array(Y)

In [35]:
n = len(Y)
print n

6258


Всего в обучающей выборке n пар.
Посмотрим на первые 10 значений каждого вектора:

In [36]:
print X[:10]
print Y[:10]

[185 185 185  28  28  28  28 336 336 336]
[ 28 336  39 185 336  39 110  28  39 185]


Всё в порядке - код сработал верно.

Каждое слово можно рассматривать как отдельный класс. Представим слово как вектор длины voc_size. На позиции, равной индексу слова в словаре, будет стоять 1, а на всех остальнх позициях - 0 (заметим, что при этом сумма чисел в векторе равна единице и у нас получается аналогия стопроцентной вероятности). Подобное представление получило название one-hot encoding.

In [130]:
def one_hot_encoder(y):
    
    n = len(y)
    hot = np.zeros((voc_size, n))
    
    for i in range(n):
        hot[y[i], i] = 1
    
    hot = hot.astype(np.int64)
    
    return hot

In [131]:
Y_hot = one_hot_encoder(Y)

На этом мы заканчиваем препроцессинг данных и переходим собственно к построению модели. Однако стоит отметить, что мы сделали далеко не всё, что можно было сделать. Напрмер, слишком часто встречающиеся слова ("a", "the", "and" etc) можно не принимать в рассчёт, поскольку они могут попасть в контекстное окно практически с любым словом. Аналогично с очень редко используемыми словами - для них будет слишком маленькая выборка контекстных слов, по которой нельзя построить хорошую зависимость. Но чтобы убирать такие слова, пришлось бы подбирать ещё два гиперпараметра - предельные (верхняя и нижняя) частоты встречаемости. Я не стала этого делать.

## Инициализация модели

На выходе работы программы мы хотим получить матрицу, задающую векторное представление для каждого слова, причём таким образом, что чем чаще пара слов встречаются в одном контекстном окне, тем ближе друг к другу точки, описываемые этими векторами. Следовательно, у нас появился ещё один гиперпараметр - размерность пространства слов (emb_size, от embedding - прикрепляющий, запечатляющий). Если взять параметр emb_size слишком маленьким, то модель будет плохо описывать реальность, а если слишком большим - долго обучаться.

In [39]:
emb_size = 10

Вернемся к матрице, получаемой на выходе. Она имеет размеры (voc_size, emb_size), чтобы каждому слову в словаре соответствовал вектор. Для начала инициализируем эту матрицу случайными значениями:

In [40]:
word_emb = np.random.sample((voc_size, emb_size))

Также наша модель будет иметь один скрытый слой - dense_layer. На вход этому слою будут подаваться веторные представления слов из вектора X, которых всего n штук. То есть на вход dense_layer получает матрицу размера (emb_size, n). На выходе скрытый слой должен давать вероятности того, что каждое заданное слово (которых всего n) окажется в паре с каждым словом из словаря (а их voc_size штук) - таким образом на выходе мы ожидаем матрицу размера (voc_size, n). Так как в скрытом слое будет проходить операция матричного умножения, он тоже должен иметь размеры (voc_size, emb_size). 

Инициализируем dense_layer случайными значениями:

In [41]:
dense_layer = np.random.sample((voc_size, emb_size))

Заведём словарь для хранения тренируемых параметров:

In [110]:
weights = {}
weights["word_emb"] = word_emb
weights['dense_layer'] = dense_layer
weights["X"] = X

## Прямое распространение ошибки

Первый этап в каждом цикле обучения - прямое распространение ошибки (англ. forward propagation). 

На вход программы подаются слова, которые прежде всего нужно превратить в векторы. Векторные представления слов хранятся в массиве word_emb словаря weights:

In [113]:
def vector_representations(X, weights=weights):
    
    word_emb = weights['word_emb']
    word_vec = word_emb[X.flatten(), :].T
    weights['word_vec'] = word_vec
    
    return word_vec

Написанная выше функция возвращает матрицу, у которой в каждой строке закодированы emb_size-мерными векторами подаваемые на вход слова. Применим функцию к ветору целевых слов X и посмотрим на первые 5 элементов получившейся матрицы:

In [114]:
word_vec_0 = vector_representations(X, weights=weights)
print word_vec_0[:5]

[[0.2924399  0.2924399  0.2924399  ... 0.99513672 0.99513672 0.99513672]
 [0.47131404 0.47131404 0.47131404 ... 0.67305381 0.67305381 0.67305381]
 [0.72300482 0.72300482 0.72300482 ... 0.36731023 0.36731023 0.36731023]
 [0.72484276 0.72484276 0.72484276 ... 0.59013117 0.59013117 0.59013117]
 [0.91495268 0.91495268 0.91495268 ... 0.34057055 0.34057055 0.34057055]]


Всё работает.

Следующий шаг - перемножить получившуюся матрицу с тренируемой матрицей dense_layer.

In [46]:
def dense_layer_multiplication(word_vec, weights=weights):
    
    dense_layer = weights['dense_layer']
    Z = np.dot(dense_layer, word_vec)
    
    return dense_layer, Z

Посмотрим на результат работы функции:

In [48]:
dense_layer_0, Z_0 = dense_layer_multiplication(word_vec_0, weights=weights)
print Z_0[:5]

[[3.09360033 3.09360033 3.09360033 ... 3.95876509 3.95876509 3.95876509]
 [2.24916844 2.24916844 2.24916844 ... 3.18112802 3.18112802 3.18112802]
 [3.44456342 3.44456342 3.44456342 ... 5.25263275 5.25263275 5.25263275]
 [1.72565389 1.72565389 1.72565389 ... 2.98144892 2.98144892 2.98144892]
 [1.49313057 1.49313057 1.49313057 ... 1.60964818 1.60964818 1.60964818]]


И, наконец, финальный шаг в forward propagation - применить к полученной матрице функцию softmax. Это приведет к тому, что каждая координата матрицы будет лежать на отрезке [0, 1], отношения между координатами одной строки не изменятся, а построчные суммы координат будут равняться единице.

In [68]:
def softmax(Z):
    return np.exp(Z)/np.sum(np.exp(Z), axis=0)

In [76]:
softmax_out_0 = softmax(Z_0)

print softmax_out_0[0]
print np.sum(softmax_out_0[:], axis=0)[0]

[0.00284139 0.00284139 0.00284139 ... 0.00263078 0.00263078 0.00263078]
0.9999999999999999


## Обратное распространение ошибки

На этапе backward propagation происходит подсчёт ошибки и правка тренируемых параметров. Для подсчёта ошибки возьмём функцию Cross Entropy Loss - её обычно используют для вероятностных параметров.

In [96]:
def cross_entropy(Z, y):
  
    y_1 = y.argmax(axis=1)
    p = softmax(Z)
    m = y_1.shape[0]
    log_likelihood = np.log(p[range(m),y_1])
    loss = -np.sum(log_likelihood)/m
    
    return loss

Посмотрим, какая ошибка получилась у нас после первой итерации forward propagation:

In [97]:
loss_0 = cross_entropy(Z_0, Y_hot)
print loss_0

6.266295757386325


Ошибка получилась неприемлемо большой, что не удивительно - параметры для первой итерации подбирались случайным образом.

На этапе backward propagation будем определять, в каком направлении ошибка убывает быстрее всего, после чего будем двигаться туда. Это направление задаёт антиградиент.

Теперь перйдём к самой функции обратного распространения ошибки:

In [168]:
def backward_propagation(Y_hot, softmax_out, weights, learning_rate=0.01):
    
    
    dL_dZ = softmax_out - Y_hot             # Насколько предсказанные вероятности
                                            # отличаются от истинного значения
    
    
    word_vec = weights['word_vec']           # Извлекаем из словаря нужные нам параметры
    dense_layer = weights['dense_layer']
    X = weights['X']
    word_emb = weights['word_emb']
    
    n = word_vec.shape[1]
    
    dL_d_dense_layer = (1 / n) * np.dot(dL_dZ, word_vec.T)       # Считаем градиенты весов  
    dL_dword_vec = np.dot(dense_layer.T, dL_dZ)
    
    
    gradients = dict()                                      # Сохраним посчитанные градинты
    gradients['dL_dZ'] = dL_dZ
    gradients['dL_d_dense_layer'] = dL_d_dense_layer
    gradients['dL_dword_vec'] = dL_dword_vec
    
    word_emb = weights['word_emb'] 
    word_emb[X.flatten(), :] -= dL_dword_vec.T * learning_rate                 # "Сдвигаем" значения весов на антиградиент
    weights['dense_layer'] -= learning_rate * gradients['dL_d_dense_layer']    # Обновили веса модели
                                                                               # (гиперпараметр learning_rate

## Тренировка модели

Теперь, когда все необходимые функции написаны, можно перейти к тренировке модели. Вспомним, какие у нас есть данные:

    X - индексы первых в паре слов
    Y - индексы вторых в паре слов
    Y_hot - вторые в паре слов, записанные с помощью one-hot code
    emb_size = 10 - размерность пространства векторов
    voc_size - количество слов в словаре

Зададим количество эпох, то есть то, за сколько итераций мы будем проводить обучение:

In [173]:
epochs = 10

(По-хорошему, следует делать порядка 1000 итераций, но с таким числом циклов программа работает невыносимо долго)

Запустим процесс обучения:

In [174]:
word_vec = weights['word_vec']


for epoch in range(epochs):
         
        dense_layer, Z = dense_layer_multiplication(word_vec, weights=weights)
        softmax_out = softmax(Z)
        
        backward_propagation(Y_hot, softmax_out, weights, learning_rate=0.01)
    
        cost = cross_entropy(softmax_out, Y_hot) 

## Проверка модели

In [200]:
import scipy

In [201]:
from scipy.spatial import distance

Ниже представлена функция, которая для заданного слова подбираем наиболее на него похожее (по мнению программы, естественно). Чем меньше косинусное расстояние, тем более семантически близки слова.

In [202]:
def similarity(word_emb, index):
    
    min = 10
    nearest_index = -10
    
    for i in range(voc_size):
        if scipy.spatial.distance.cosine(word_emb[i], word_emb[index]) < min and i != index:
            
            min = scipy.spatial.distance.cosine(word_emb[i], word_emb[index])
            nearest_index = i
            
    print d_indexes[index], d_indexes[nearest_index]

Посмотрим на наиболее близкое слово, подобранное каждому слову из словаря:

In [204]:
for i in range(voc_size):
    similarity(word_emb, i)

all s
impulse there
forty band
grateful they
go wondered
voids greatest
seemed tracts
dreadful accomplishing
shackles they
religion surest
concerned hollis
to drearier
chewing profanity
under trust
melon venture
suffered poor
regalia gave
scriptural weeks
feebly an
presently time
very vacation
horror desperation
diary stolen
surprise over
awful such
condition desperation
entire abroad
did no
joined withdrawing
dis nor
neighborhood tom
sensation everywhere
second parents
street lapsed
tempest claps
depression too
even found
deeply con
above made
new go
ammunition an
ever however
public overwhelming
body upon
melancholy deathbed
remem realizing
mem thing
funeral forbearance
never grateful
hours little
alone dreadful
re happened
change worn
wait took
boy no
withdrawing joined
study fixed
misery hands
frazer sadly
thatcher did
everybody actual
from afterward
would disgusted
two boys
by three
few ill
doubt on
injury failure
more old
glass hollis
everywhere sensation
broke twenty
doom relaps

Видно, что, хотя далеко не всегда слова получаются близкими по смыслу, среди получившихся пар есть весьма неплохо подобранные: "suffered-poor", "presently-time", "melancholy-deathbed", "desperation-hoping", "weeks-night", "once-forever", "bug-insect" etc (небольшая ремарка: перед начальной инициализацией параметров я забыла поставить randomstate, поэтому при повторном запуске программы результаты, вероятно, будут немного отличаться).

Чтобы улучшить результат ещё сильнее, необходимо подобрать подходящие гиперпараметры: число циклов (epochs), размер контекстного окна (window_size), размерность векторного пространства (emb_size). Если увеличить объём текста или уменьшить объём словаря, это также положительно скажется на результате работы программы.

## Заключение

На этом я заканчиваю свою работу. Надеюсь, вам понравилось.

Спасибо за внимание и терпение!

## Литература (точнее, ссылки)

https://habr.com/ru/company/ods/blog/329410/

https://neurohive.io/ru/osnovy-data-science/osnovy-nejronnyh-setej-algoritmy-obuchenie-funkcii-aktivacii-i-poteri/

https://towardsdatascience.com/word2vec-from-scratch-with-numpy-8786ddd49e72

https://deepnotes.io/softmax-crossentropy#cross-entropy-loss